In [3]:
import torch
import torch.nn.utils.prune as prune 
import numpy as np
import plotly.graph_objects as go 
import pandas as pd 
import torchvision
from torch.autograd import Variable


In [4]:
check_gpu = torch.cuda.is_available()
check_gpu

False

In [5]:
# check_devices = torch.cuda.get_device_name(0)
# check_devices

In [6]:
crypto_name = 'apeusd'
#example_ohlc_df = pd.DataFrame({"Time": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Open": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"High": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Low": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Close": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

ohlc_df = pd.read_csv('historical_price/' + crypto_name + '.csv')
ohlc_df.index = pd.to_datetime(ohlc_df['time'], unit = 'ms')

#ohlc_df.index = ohlc_df.index.tz_localize('UTC').tz_convert('US/Eastern')

In [12]:
price_dataset = ohlc_df.close[-1000:]
price_dataset.head()

time
2022-07-14 07:18:00    4.4822
2022-07-14 07:19:00    4.4764
2022-07-14 07:21:00    4.4792
2022-07-14 07:23:00    4.4885
2022-07-14 07:24:00    4.4817
Name: close, dtype: float64

In [15]:
price_dataset_x = price_dataset[500:]
price_dataset_y = price_dataset[-700:-500]
print(len(price_dataset_y))

200


In [17]:
price_test_x = price_dataset[-500:-400]
price_test_y = price_dataset[-400:-300]

In [9]:
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, seq_len=10):
        self.X = X
        self.y = y
        self.seq_len = seq_len
    def __len__(self):
        return self.X.__len__() - (self.seq_len - 1)

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index+self.seq_len-1])

In [18]:
train_dataset = TimeSeriesDataset(price_dataset_x, price_dataset_y, seq_len=10)
test_dataset = TimeSeriesDataset(price_test_x, price_test_y, seq_len=10)

In [16]:
seq_len = 10

In [21]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=seq_len, 
    shuffle=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=seq_len, 
    shuffle=False)

In [23]:
class LSTM_Model(torch.nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, embedding_length):
        super(LSTM_Model, self).__init__()
        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(embedding_length, hidden_size) # Our main hero for this tutorial
        self.label = torch.nn.Linear(hidden_size, output_size)
        
    def forward(self, input_sentence, batch_size=None):
        input = self.word_embeddings(input_sentence) 
        input = input.permute(1, 0, 2) 
        if batch_size is None:
            h_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) 
            c_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) 
        else:
            h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
        output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        final_output = self.label(final_hidden_state[-1]) 
        
        return final_output
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
crypto_price_model = LSTM_Model(batch_size=seq_len, output_size=1, hidden_size=64, embedding_length=1)

if torch.cuda.is_available():
    crypto_price_model.cuda()
crypto_price_model.to(device)

loss_function = torch.nn.CrossEntropyLoss() # Criterion, I believe CrossEntropyLoss has SoftMax built-in so I used it here, this is why my last layer doesn't have a softmax
optimizer  = torch.optim.Adam(crypto_price_model.parameters(), lr=0.001) # used adaptive moment estimation to optimize the model

cpu


In [24]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text = batch.text[0]
        target = batch.label
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] is not 32): 
            continue
        optim.zero_grad()
        prediction = model(text)
        loss = loss_fn(prediction, target)
        wandb.log({"Training Loss": loss.item()})
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        wandb.log({"Training Accuracy": acc.item()})
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch.text[0]
            if (text.size()[0] is not 32):
                continue
            target = batch.label
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            prediction = model(text)
            loss = loss_fn(prediction, target)
            wandb.log({"Evaluation Loss": loss.item()})
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            wandb.log({"Evaluation Accuracy": acc.item()})
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)

<>:20: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:20: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\Austin\AppData\Local\Temp\ipykernel_276\2592159120.py:20: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (text.size()[0] is not 32):
C:\Users\Austin\AppData\Local\Temp\ipykernel_276\2592159120.py:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (text.size()[0] is not 32):


In [ ]:
torch.save(crypto_price_model.state_dict(), f'{crypto_name}_model_weights.pth')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
crypto_price_model.load_state_dict(torch.load(f'{crypto_name}_model_weights.pth', map_location=torch.device(device)))